In [96]:
def createDataSet():
    #the last column is used for the labels
    dataSet = [[1, 1, 'yes'],
               [1, 1, 'yes'],
               [1, 0, 'no'],
               [0, 1, 'no'],
               [0, 1, 'no']]
    labels = ['no surfacing','flippers']
    #change to discrete values
    return dataSet, labels

In [97]:
myDataSet, myLabels = createDataSet()

In [98]:
myDataSet, myLabels

([[1, 1, 'yes'], [1, 1, 'yes'], [1, 0, 'no'], [0, 1, 'no'], [0, 1, 'no']],
 ['no surfacing', 'flippers'])

In [99]:
from math import log
def calcShannonEntropy(dataSet):
    numEntries = len(dataSet)
    labelCounts = {}
    for featVec in dataSet:
        currLabel = featVec[-1]
        if currLabel not in labelCounts.keys():
            labelCounts[currLabel] = 0
        labelCounts[currLabel] += 1
    entropy = 0.0
    for key in labelCounts:
        prob = float(labelCounts[key]) / numEntries
        entropy -= prob * log(prob, 2)
    return entropy

In [100]:
calcShannonEntropy(myDataSet)

0.9709505944546686

In [101]:
def splitDataSet(dataSet, axis, value):
    splitSet = []
    for vec in dataSet:
        if vec[axis] == value:
            splitSet.append(vec[:axis] + vec[axis+1:])
    return splitSet

In [102]:
myDataSet

[[1, 1, 'yes'], [1, 1, 'yes'], [1, 0, 'no'], [0, 1, 'no'], [0, 1, 'no']]

In [103]:
splitDataSet(myDataSet, 1, 0)

[[1, 'no']]

In [104]:
splitDataSet(myDataSet, 1, 1)

[[1, 'yes'], [1, 'yes'], [0, 'no'], [0, 'no']]

In [105]:
def chooseBestFeatureToSplit(dataSet):
    #the last column is used for the labels
    numFeatures = len(dataSet[0]) - 1
    baseEntropy = calcShannonEntropy(dataSet)
    bestGain = 0.0
    bestFeature = -1
    for i in range(numFeatures):
        # get a set of unique values for the feature
        values = set([v[i] for v in dataSet]) 
        newEntropy = 0.0
        for val in values:
            splitSet = splitDataSet(dataSet, i, val)
            prob = len(splitSet) / float(len(dataSet))
            newEntropy += prob * calcShannonEntropy(splitSet)
        # information gain is the reduction in entropy
        infoGain = baseEntropy - newEntropy
        if (infoGain > bestGain):
            bestGain = infoGain
            bestFeature = i
    return bestFeature

In [106]:
chooseBestFeatureToSplit(dataSet)

0

In [107]:
def majorityCnt(classList):
    classCount={}
    for vote in classList:
        if vote not in classCount.keys(): classCount[vote] = 0
        classCount[vote] += 1
    sortedClassCount = sorted(classCount.iteritems(), key=operator.itemgetter(1), reverse=True)
    return sortedClassCount[0][0]

def createTree(dataSet, labels):
    labels = labels.copy()
    classList = [example[-1] for example in dataSet]
    if classList.count(classList[0]) == len(classList): 
        return classList[0]#stop splitting when all of the classes are equal
    if len(dataSet[0]) == 1: #stop splitting when there are no more features in dataSet
        return majorityCnt(classList)
    bestFeat = chooseBestFeatureToSplit(dataSet)
    bestFeatLabel = labels[bestFeat]
    myTree = {bestFeatLabel:{}}
    del(labels[bestFeat])
    featValues = [example[bestFeat] for example in dataSet]
    uniqueVals = set(featValues)
    for value in uniqueVals:
        subLabels = labels[:]       #copy all of labels, so trees don't mess up existing labels
        myTree[bestFeatLabel][value] = createTree(splitDataSet(dataSet, bestFeat, value),subLabels)
    return myTree                            

In [108]:
myTree = createTree(myDataSet, myLabels)

In [109]:
myTree

{'no surfacing': {0: 'no', 1: {'flippers': {0: 'no', 1: 'yes'}}}}

In [110]:
myLabels

['no surfacing', 'flippers']